### Preparation 

In [ ]:
import json
with open('secret_tokens.json') as f:
    tokens_dict = json.load(f)

### CodeLLama

In [ ]:
import importlib
importlib.reload(clw)

In [ ]:
import codellama_wrapper as clw
codellama_trial = clw.CodellamaEval(model_name = "codellama/CodeLlama-7b-hf", gpu_id=2)


In [ ]:
codellama_trial.generate()
file_name = codellama_trial.model_process()

In [ ]:
codellama_trial.evaluate()

In [ ]:
# If we have a json with the data, we can run just the eval and skip the generation
import importlib
importlib.reload(clw)
clt = clw.CodellamaEval(model_name = None, method_dict_location = 'output_multi-humaneval_CodeLlama-7b-hf.json')

### Refact

In [ ]:
# I've implemented new multi_evaluate routine that does everything in one click.
# Unlike generic evaluate, it runs code generation several times and evaluates several iterations. In the end this yields pass@1 averaged over several runs
# One generation + eval takes circa 15 minutes on one 4090 GPU
import causallm_wrapper as clmw
refact = clmw.CausalLMEval(gpu_id = 0)
refact.multi_evaluate(iterations = 5)

In [ ]:
#For CausalLM wrapper, I've also added loading from checkpoint, which can be used as follows
import causallm_wrapper as clmw
refact2 = clmw.CausalLMEval(model_checkpoint = './checkpoint/stack_tuned.ckpt', gpu_id = 0)
refact2.multi_evaluate(iterations = 5)

### Dataset fiddling (for dev purposes, you are unlikely to need that)

In [ ]:
import json
with open('output_multi-humaneval_Refact-1_6B-fim.json') as f:
    results_dict = json.load(f)

In [ ]:
import jsonlines
with jsonlines.open('output_multi-humaneval_Refact-1_6B-fim.jsonl_results.jsonl') as reader:
    for obj in reader:
        print(obj['task_id'])
        print(obj['completion'])
        print(obj['result'])
        print('-'*80)

In [ ]:
for i, item in enumerate(refact.raw_method_list):
    print(i)
    print(item)
    print('\n')
    print('-'*80)

In [ ]:
with open('output_multi-humaneval_Refact-1_6B-fim.jsonl_results.jsonl') as f:
    lines = f.read().splitlines()
    line_dicts = [json.loads(line) for line in lines]
df = pd.DataFrame(line_dicts)
pretty_print(df)

In [ ]:
import jsonlines
with jsonlines.open(file_name) as reader:
    for obj in reader:
        print(obj['completion'])
        print('----')

In [ ]:
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [ ]:
import pandas as pd
import json